<a href="https://colab.research.google.com/github/slunara/corporateProject/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Revenue Forecast

In [1]:
import numpy as np
import pandas as pd


def compute_customer_forecast(n_months, traffic, prob_prospect_generation,
                              prob_prospect_conversion,
                              prob_direct_customer_conversion, retention_prob, prob_existing_clients_conversion,
                              existing_customers):

    prospects = traffic * prob_prospect_generation

    rows, cols = np.meshgrid(np.arange(n_months), np.arange(n_months), indexing="ij")

    M_conversion_matrix_ops = np.where(cols <= rows, prob_prospect_conversion[rows-cols], 0)

    new_customers_from_prospects = M_conversion_matrix_ops @ prospects

    new_customers_direct = traffic * prob_direct_customer_conversion

    M_retention_matrix_ops = np.where(cols <= rows, retention_prob[rows-cols], 0)

    retained_customers_from_prospects = M_retention_matrix_ops @ new_customers_from_prospects

    retained_customers_direct = M_retention_matrix_ops @ new_customers_direct

    existing_customers_vector = np.full(n_months, existing_customers * prob_existing_clients_conversion)

    total_existing_customers = existing_customers_vector + retained_customers_from_prospects + retained_customers_direct

    forecast_df = pd.DataFrame({
    "prospects": prospects,
    "new_customers_from_prospects": new_customers_from_prospects,  # Sum across rows
    "new_customers_direct": new_customers_direct,
    "total_existing_customers": total_existing_customers
    }, index=[f"Month {i+1}" for i in range(n_months)])

    return forecast_df

def compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df):

    total_revenue_df = pd.DataFrame(index=local_customer_forecast_df.index)

    total_revenue_df['revenue_local_new_from_prospects'] = local_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['local_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_local_new_from_direct'] = local_customer_forecast_df['new_customers_direct']*avg_ticket_df['local_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_local_existing'] = local_customer_forecast_df['total_existing_customers']*avg_ticket_df['local_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_local_total'] = total_revenue_df['revenue_local_new_from_prospects']+total_revenue_df['revenue_local_new_from_direct']+total_revenue_df['revenue_local_existing']

    total_revenue_df['revenue_tourist_new_from_prospects'] = tourist_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['tourist_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_tourist_new_from_direct'] = tourist_customer_forecast_df['new_customers_direct']*avg_ticket_df['tourist_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_tourist_existing'] = tourist_customer_forecast_df['total_existing_customers']*avg_ticket_df['tourist_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_tourist_total'] = total_revenue_df['revenue_tourist_new_from_prospects']+total_revenue_df['revenue_tourist_new_from_direct']+total_revenue_df['revenue_tourist_existing']

    total_revenue_df['revenue_total'] = total_revenue_df['revenue_local_total']+total_revenue_df['revenue_tourist_total']
    return total_revenue_df

In [2]:
# ------------------------------ #
# Example usage for Local & Tourist Customers
# ------------------------------ #

n_months = 6
traffic = np.array([1000, 1200, 1100, 1300, 1250, 1400])  # traffic
prob_prospect_generation = 0.30
prob_prospect_conversion = np.array([0.3, 0.2, 0.1, 0, 0, 0])  # 30% of traffic converts into prospects

local_prob_direct_customer_conversion = 0.1  # 10% of traffic converts directly into customers
tourist_prob_direct_customer_conversion = 0.05

local_retention_prob = np.array([1.0, 0.7, 0.5, 0.3, 0.2, 0.1])  # Retention probability for new customers
tourist_retention_prob = np.array([0, 0.7, 0.5, 0.3, 0.2, 0.1])  # Retention probability for new customers

local_prob_existing_clients_conversion = 0.03
tourist_prob_existing_clients_conversion = 0.01

existing_local_customers = 5000  # Existing local customers before the forecast
existing_tourist_customers = 1000  # Existing tourist customers before the forecast

local_avg_ticket_new_from_prospects = 50
local_avg_ticket_new_direct = 60
local_avg_ticket_existing = 40

tourist_avg_ticket_new_from_prospects = 0
tourist_avg_ticket_new_direct = 80
tourist_avg_ticket_existing = 45

avg_ticket_df = pd.DataFrame({
    "local_avg_ticket_new_from_prospects": [local_avg_ticket_new_from_prospects],
    "local_avg_ticket_new_direct": [local_avg_ticket_new_direct],
    "local_avg_ticket_existing": [local_avg_ticket_existing],
    "tourist_avg_ticket_new_from_prospects": [tourist_avg_ticket_new_from_prospects],
    "tourist_avg_ticket_new_direct": [tourist_avg_ticket_new_direct],
    "tourist_avg_ticket_existing": [tourist_avg_ticket_existing]
})


# Compute forecasts for Local Customers
local_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, prob_prospect_generation,prob_prospect_conversion,
    local_prob_direct_customer_conversion, local_retention_prob, local_prob_existing_clients_conversion,
    existing_local_customers
)

# Compute forecasts for Tourist Customers
tourist_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, prob_prospect_generation,  prob_prospect_conversion,
    tourist_prob_direct_customer_conversion, tourist_retention_prob, tourist_prob_existing_clients_conversion,
    existing_tourist_customers
)

total_revenue=compute_revenue_forecast(local_customer_forecast_df,tourist_customer_forecast_df,avg_ticket_df)

# Sensitivity Analysis

Identify the most influential variable for revenue

In [3]:
variables = {
    "local_prob_prospect_generation": prob_prospect_generation,
    "local_prob_prospect_conversion": prob_prospect_conversion,
    "local_prob_direct_customer_conversion": local_prob_direct_customer_conversion,
    "local_prob_existing_clients_conversion": local_prob_existing_clients_conversion,
    "local_retention_prob":local_retention_prob,

    "tourist_prob_direct_customer_conversion":tourist_prob_direct_customer_conversion,
    "tourist_prob_existing_clients_conversion": tourist_prob_existing_clients_conversion,
    "tourist_retention_prob":tourist_retention_prob
}

In [5]:
# Define baseline total revenue
baseline_total_revenue = total_revenue['revenue_total'].sum()
# Store sensitivity results
sensitivity_results = {}

# Loop through each variable to test its sensitivity
for var in variables.keys():
    # Create a copy of the original variables (fresh start each time)

    modified_variables = {k: v.copy() if isinstance(v, np.ndarray) else v for k, v in variables.items()}


    # Apply a 1% increase ONLY to the current variable
    modified_variables[var] = variables[var] * 1.01  # Increase by 1%


    # Compute forecasts using modified variables
    local_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        modified_variables["local_prob_prospect_generation"],
        modified_variables["local_prob_prospect_conversion"],
        modified_variables["local_prob_direct_customer_conversion"],
        modified_variables["local_retention_prob"],
        modified_variables["local_prob_existing_clients_conversion"],
        existing_local_customers
    )

    tourist_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        prob_prospect_generation,
        prob_prospect_conversion,
        modified_variables["tourist_prob_direct_customer_conversion"],
        modified_variables["tourist_retention_prob"],
        modified_variables["tourist_prob_existing_clients_conversion"],
        existing_tourist_customers
    )

    # Compute new total revenue
    new_total_revenue_df = compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df)

    # Extract sum of revenue
    new_total_revenue = new_total_revenue_df['revenue_total'].sum()


    # Calculate percentage impact
    sensitivity_results[var] = ((new_total_revenue - baseline_total_revenue) / baseline_total_revenue) * 100

# Convert results to DataFrame and display
sensitivity_df = pd.DataFrame.from_dict(sensitivity_results, orient='index', columns=["% Impact on Total Revenue"])
sensitivity_df = sensitivity_df.sort_values(by="% Impact on Total Revenue", ascending=False)

In [6]:
sensitivity_df

,% Impact on Total Revenue
local_retention_prob,0.390769
local_prob_prospect_generation,0.381516
local_prob_prospect_conversion,0.381516
local_prob_direct_customer_conversion,0.265968
tourist_retention_prob,0.179712
tourist_prob_direct_customer_conversion,0.119536
local_prob_existing_clients_conversion,0.091889
tourist_prob_existing_clients_conversion,0.006892


# Revenue GAP Forecast vs Budget

In [12]:
forecast=baseline_total_revenue

In [16]:
(0.01*increment_Needed/sensitivity_df)*100

,% Impact on Total Revenue
local_retention_prob,0.053719
local_prob_prospect_generation,0.055021
local_prob_prospect_conversion,0.055021
local_prob_direct_customer_conversion,0.078925
tourist_retention_prob,0.116807
tourist_prob_direct_customer_conversion,0.175609
local_prob_existing_clients_conversion,0.228444
tourist_prob_existing_clients_conversion,3.045926


In [ ]:
def compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df):

    total_revenue_df = pd.DataFrame(index=local_customer_forecast_df.index)

In [ ]:
def sensivility_analysis (baseline_total_revenue,variables):
  return sensitivity_df

In [17]:
def compute_new_forecast(sales_budget,sensitivity_df,baseline_total_revenue):
    increment_Needed=(sales_budget/baseline_total_revenue)-1
    incrementPerAction= (0.01*increment_Needed/sensitivity_df)*100
    incrementPerAction = incrementPerAction.rename(columns={"% Impact on Total Revenue": "Required Percentage Change"})

    return incrementPerAction


In [19]:
compute_new_forecast(sales_budget,sensitivity_df,baseline_total_revenue)

,Required Percentage Change
local_retention_prob,0.053719
local_prob_prospect_generation,0.055021
local_prob_prospect_conversion,0.055021
local_prob_direct_customer_conversion,0.078925
tourist_retention_prob,0.116807
tourist_prob_direct_customer_conversion,0.175609
local_prob_existing_clients_conversion,0.228444
tourist_prob_existing_clients_conversion,3.045926


# KPI re-calculation

## not done yet

In [266]:
def compute_kpi(local_new_direct,traffic,local_new_from_prospects,local_existing,existing_local_customers,tourist_new_direct,tourist_existing,existing_tourist_customers):
    local_new_closing_ratio=local_new_direct.sum().values[0]/traffic.sum()
    prospect_closing_ratio=local_new_from_prospects.sum().values[0]/traffic.sum()
    local_come_back=local_existing.sum().values[0]/(existing_local_customers+local_existing.sum().values[0])
    tourist_new_closing_ratio=tourist_new_direct.sum().values[0]/traffic.sum()
    tourist_come_back=tourist_existing.sum()/(existing_tourist_customers+tourist_existing.sum().values[0])
    kpi_df=pd.DataFrame([local_new_closing_ratio,prospect_closing_ratio,local_come_back,tourist_new_closing_ratio, tourist_come_back])
    return kpi_df


In [267]:
def compute_kpi(traffic,existing_local_customers,existing_tourist_customers,total_revenue_df):
    local_new_closing_ratio=total_revenue_df['revenue_local_new_from_direct'].sum().values[0]/traffic.sum()
    prospect_closing_ratio=total_revenue_df['revenue_local_new_from_prospects'].values[0]/traffic.sum()
    local_come_back=total_revenue_df['revenue_local_existing'].values[0]/(existing_local_customers+total_revenue_df['revenue_local_new_from_direct'].values[0]+)
    tourist_new_closing_ratio=tourist_new_direct.sum().values[0]/traffic.sum()
    tourist_come_back=tourist_existing.sum()/(existing_tourist_customers+tourist_existing.sum().values[0])

    kpi_df=pd.DataFrame([local_new_closing_ratio,prospect_closing_ratio,local_come_back,tourist_new_closing_ratio, tourist_come_back])
    return kpi_df

kpi_df=compute_kpi(traffic,existing_local_customers,existing_tourist_customers,total_revenue_df)

SyntaxError: invalid syntax (<ipython-input-267-f915514df4a4>, line 4)